## 获得-处理 数据

#### 导入包

In [32]:
import numpy as np
import pandas as pd
import asyncio

from datetime import time
from CSVModifier import aggregate_high_freq_to_low, add_factors, clean_outliers
from Preprocessing import aggregate_tick_to_minute
from DataRequest import fetch_tick_data, fetch_candle_data

### 必要设置：

In [2]:
SYMBOL = 'SPY'
START_DATE = '2025-05-03'
END_DATE = '2025-05-06'
FINHUB_TOKEN = 'cvop3lhr01qihjtq3uvgcvop3lhr01qihjtq3v00'

### 获取tick数据：

In [ ]:
df_ticks = fetch_tick_data(
    symbol=SYMBOL,
    start_date=START_DATE,
    end_date=END_DATE,
    api_key=FINHUB_TOKEN,
    page_workers=10,
    day_workers=5,
    limit=25000,
    sleep_sec=0.1
)


In [ ]:
# （可选）
# df_ticks.to_csv(f"{SYMBOL}_{START_DATE}_{END_DATE}_ticks.csv", index=False)

### 处理数据：
给数据做集合，过滤

In [31]:
df_minute = aggregate_tick_to_minute(df_ticks)
df_minute = df_minute.dropna()
df_minute.columns

[Info] Dropped 1956 rows due to invalid timestamps.


Index(['timestamp', 'open', 'high', 'low', 'close', 'volume', 'dollar_volume',
       'vwap', 'tick_count', 'trade_size_mean', 'trade_size_std',
       'zero_return_count', 'price_direction_ratio', 'large_trade_count',
       'large_trade_ratio', 'large_trade_volume_ratio'],
      dtype='object')

### 请求分钟数据（可选）
可以用来做验证。看看tick 数据的行数是否对的上


In [19]:
min_interval = 1
# 请求数据
df_finhub_agg = fetch_candle_data(
    symbol= SYMBOL,
    start_date=START_DATE,
    end_date=END_DATE,
    interval=min_interval, # 单位为分钟
    token=FINHUB_TOKEN,
    chunk_days=25,
    max_workers=10
)


SPY 1-min K 线块: 100%|██████████| 1/1 [00:00<00:00,  5.48it/s]


### 验证数据行数（可选）
可以在这里比较数据的准确性等

In [ ]:
# 定义交易时间
start_time = time(9, 30)
end_time   = time(16, 0)

# 构造掩码：9:30–16:00
df_minute_masked = df_minute[df_minute['timestamp'].dt.time.between(start_time, end_time)]
df_finhub_agg_masked = df_finhub_agg[df_finhub_agg['timestamp'].between(start_time, end_time)]

df_minute_masked.shape, df_finhub_agg_masked.shape

#### 

In [ ]:
path_lo = f"{SYMBOL}_{START_DATE}_{END_DATE}_{LOFREQ}m_raw.csv"
path_hi = f"{SYMBOL}_{START_DATE}_{END_DATE}_{HIFRREQ}m_raw.csv"
df_loFreq.to_csv(path_lo, index=False)
df_hiFreq.to_csv(path_hi, index=False)

In [ ]:
# 检查数据
df_loFreq.shape
# df_hiFreq.tail(2)

In [ ]:
# 把高频转换成低频集合
df_hiFreq_agg = aggregate_high_freq_to_low (df_hiFreq, freq='5min')

# 计算因子
df_loFreq_factored = add_factors (df_loFreq)

# # 拼接
df_merged = pd.merge (df_loFreq_factored, df_hiFreq_agg, how='left', on='timestamp')
df_merged = clean_outliers(df_merged, z_thresh=10, show_msg=True)

In [ ]:
# 检查数值
# df_merged[df_merged['close_volatility'] == 0]

df_merged.shape

In [ ]:
import re
path = f"{SYMBOL}_{START_DATE}_{END_DATE}_{LOFREQ}m.csv"
# path = re.sub(r'[^\w\-]', '', path)
df_merged.to_csv(path, index=False)